In [ ]:
import datasets

In [4]:
from datasets import Dataset

# Example documents
documents = [
    {"text": "The quick brown fox jumps over the lazy dog."},
    {"text": "All work and no play makes Jack a dull boy."},
    {"text": "To be, or not to be, that is the question."},
    {"text": "Budumba - this is a rare creature that inhabits the forests of the Northern Lands. It is named after the Native Americans who named it 'Budumba' because of its large head. It uses it's head to crack rocks and break up large trees to find food. It is a common sight in the forest."}
]

# Create a dataset from the documents
dataset = Dataset.from_dict({"documents": [doc["text"] for doc in documents]})

print(dataset)

Dataset({
    features: ['documents'],
    num_rows: 4
})


In [10]:
from transformers import DPRContextEncoder, DPRContextEncoderTokenizer, DPRQuestionEncoder, DPRQuestionEncoderTokenizer
import torch
with torch.set_grad_enabled(False):
    ctx_encoder = DPRContextEncoder.from_pretrained("facebook/dpr-ctx_encoder-single-nq-base")
    ctx_tokenizer = DPRContextEncoderTokenizer.from_pretrained("facebook/dpr-ctx_encoder-single-nq-base")

    ds = dataset.map(lambda example: {'embeddings': ctx_encoder(**ctx_tokenizer(example["documents"], return_tensors="pt"))[0][0].numpy()})
    ds.add_faiss_index(column="embeddings")

Some weights of the model checkpoint at facebook/dpr-ctx_encoder-single-nq-base were not used when initializing DPRContextEncoder: ['ctx_encoder.bert_model.pooler.dense.bias', 'ctx_encoder.bert_model.pooler.dense.weight']
- This IS expected if you are initializing DPRContextEncoder from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DPRContextEncoder from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'DPRQuestionEncoderTokenizer'. 
The class this function is called from is 'DPRContextEncoderTokeniz

In [2]:
train_test_split = dataset.train_test_split(test_size=0.2)

print(train_test_split)

DatasetDict({
    train: Dataset({
        features: ['documents'],
        num_rows: 2
    })
    test: Dataset({
        features: ['documents'],
        num_rows: 1
    })
})


In [11]:

ds.save_to_disk("my_dataset")
ds.save_faiss_index('embeddings', 'my_index.faiss')


ValueError: please remove all the indexes using `dataset.drop_index` before saving a dataset